### Automotive Sales Data Analysis Project

- Analyze automotive sales data to understand car pricing, trends, and market behavior. 
- Use SQL to clean, aggregate, and explore the dataset, answering questions like average selling price by make/model, condition vs. price, regional sales trends, and date-based patterns. 
- Builds practical SQL skills while simulating real industry analysis.

In [ ]:
-- Identify and remove duplicate VIN entries.

DELETE FROM carprices
WHERE vin in
(
    select vin from
    (
        select 
                vin,
                row_number() over(partition by vin order by year) as dup_vin
            from 
                carprices
    ) t
    where dup_vin > 1
)

In [ ]:
-- Find rows with missing values in sellingprice or mmr.

SELECT
    *
FROM
    carprices
WHERE
    sellingprice is null or mmr is null

In [ ]:
-- Replace null values in interior with “Unknown.”

SELECT
    year,
    make,
    model,
    vin,
    coalesce(interior,'Unknown')
FROM 
    carprices

In [ ]:
-- Standardize make column (e.g., unify “KIA” and “Kia”).

UPDATE 
    carprices
SET 
    make = 'KIA'
WHERE 
    make ILIKE 'Kia';

In [ ]:
-- Trim extra spaces from model and trim columns.

SELECT
    year,
    make,
    trim(model) AS model,
    trim("trim") AS trim,
    vin
FROM 
    carprices

In [ ]:
--  Creating copy

CREATE TABLE carprices_copy CLONE carprices;

/*
CREATE TABLE carprices_copy CLONE carprices;

ALTER TABLE carprices_copy 
ADD COLUMN saledate_date DATE;

UPDATE carprices_copy
SET saledate_date = TO_DATE(
  TRY_TO_TIMESTAMP_LTZ(
    saledate, 
    'DY MON DD YYYY HH24:MI:SS "GMT"TZHTZM (TZD)'
  )
);

ALTER TABLE carprices_copy 
DROP COLUMN saledate;

ALTER TABLE carprices_copy 
RENAME COLUMN saledate_date TO saledate;
*/

In [ ]:
-- Convert saledate to proper DATE format.
-- Current sample date format: Thu Feb 12 2015 01:30:00 GMT-0800 (PST)

UPDATE carprices_copy
SET saledate = TO_DATE(
  TRY_TO_TIMESTAMP_LTZ(
    saledate, 
    'DY MON DD YYYY HH24:MI:SS "GMT"TZHTZM (TZD)'
  )
);

In [ ]:
-- Extract month and year from saledate for reporting.

SELECT
    year,
    make,
    model,
    vin,
    extract(month from saledate) as sale_month,
    extract(year from saledate) as sale_year
FROM
    carprices_copy

In [ ]:
-- Flag cars with year greater than the current year.

SELECT 
    * 
FROM 
    carprices_copy
WHERE
    year > extract(year from current_date())

In [ ]:
-- Correct negative values in odometer.

SELECT
    year,
    make,
    model,
    vin,
    abs(odometer) as odometer
FROM
    carprices

In [ ]:
-- Standardize color names (e.g., “Grey” → “Gray”).

UPDATE
    carprices_copy
SET
    color = 'Gray'
WHERE
    color ILIKE 'Grey'